## Notes:

 filter image from grayscale to black and white
 make pixels that pass the threshold full white
 use gaussian rather than mean, seems to give better results
 invert so foreground will be white, background will be black
 size of a pixel neighborhood used to calculate threshold value
 constant subtracted from the mean or weighted mean
 input image, make sure to use a copy since the function will modify this image in the course of finding contours
 compress horizontal, vertical, and diagonal segments and leave only their end points

# OCR - Computer generated texts

## Import modules

In [1]:

import cv2
import numpy as np
import operator
import os
# import argparse
import easygui


## Image dataset information

In [2]:

MIN_CONTOUR_AREA = 100
RESIZED_IMAGE_WIDTH = 20
RESIZED_IMAGE_HEIGHT = 30


## Bounding rectangle

In [3]:

class ContourWithData():
    Contour = None           # contour
    boundingRect = None       
    intRectX = 0                
    intRectY = 0            
    intRectWidth = 0           
    intRectHeight = 0         
    fltArea = 0.0              

    # calculate bounding rect info
    def calculateRectTopLeftPointAndWidthAndHeight(self):
        [intX, intY, intWidth, intHeight] = self.boundingRect
        self.intRectX = intX
        self.intRectY = intY
        self.intRectWidth = intWidth
        self.intRectHeight = intHeight

    def checkIfContourIsValid(self):
        if self.fltArea < MIN_CONTOUR_AREA:
            return False        # much better validity
        return True



## Data and information inside contours

In [4]:
def main():
    allContoursWithData = []
    validContoursWithData = []

    try:
        # read in training classifications
        Classifications = np.loadtxt(
            "data-files/classifications.txt", np.float32)
    except:
        print("error, unable to open classifications.txt, exiting program\n")
        os.system("pause")
        return

    try:
        # read in training images
        FlattenedImages = np.loadtxt(
            "data-files/flattened_images.txt", np.float32)
    except:
        print("error, unable to open flattened_images.txt, exiting program\n")
        os.system("pause")
        return

    # reshape numpy array to 1d, necessary to pass to call to train
    Classifications = Classifications.reshape(
        (Classifications.size, 1))
    kNearest = cv2.ml.KNearest_create()       # instantiate KNN object
    kNearest.train(FlattenedImages, cv2.ml.ROW_SAMPLE, Classifications)
    image_picked = easygui.fileopenbox()

    inputTestingImage = cv2.imread(image_picked)
    if inputTestingImage is None:               
        print("error: image not read from file \n\n")
        os.system("pause")
        return

    imgGray = cv2.cvtColor(inputTestingImage, cv2.COLOR_BGR2GRAY)
    imgBlurred = cv2.GaussianBlur(imgGray, (5, 5), 0)                

    imgThresh = cv2.adaptiveThreshold(
        imgBlurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
    imgThreshCopy = imgThresh.copy()

    Contours, Hierarchy = cv2.findContours(imgThreshCopy,            
                                           cv2.RETR_EXTERNAL,        
                                           cv2.CHAIN_APPROX_SIMPLE)  

    for Contour in Contours:
        # instantiate a contour with data object
        contourWithData = ContourWithData()
        # assign contour to contour with data
        contourWithData.Contour = Contour
        contourWithData.boundingRect = cv2.boundingRect(
            contourWithData.Contour)     # get the bounding rect
        contourWithData.calculateRectTopLeftPointAndWidthAndHeight(
        )                    # get bounding rect info
        contourWithData.fltArea = cv2.contourArea(
            contourWithData.Contour)           # calculate the contour area
        
        # add contour with data object to list of all contours with data
        allContoursWithData.append(contourWithData)

    for contourWithData in allContoursWithData:
        if contourWithData.checkIfContourIsValid():

            # if so, append to valid contour list
            validContoursWithData.append(contourWithData)

    validContoursWithData.sort(key=operator.attrgetter(
        "intRectX"))         # sort contours from left to right

    # declare final string
    string_output = ""

    for contourWithData in validContoursWithData:

        # draw a border rect around the current char
        cv2.rectangle(inputTestingImage,              # draw rectangle on original testing image
                   
                      (contourWithData.intRectX, contourWithData.intRectY),
                      (contourWithData.intRectX + contourWithData.intRectWidth,
                       contourWithData.intRectY + contourWithData.intRectHeight),     
                      (155, 200, 50),  
                      2)                       

        imgROI = imgThresh[contourWithData.intRectY: contourWithData.intRectY + contourWithData.intRectHeight, # crop char out of threshold image
                           contourWithData.intRectX: contourWithData.intRectX + contourWithData.intRectWidth]

        # resize image, recoginition and consitent
        imgROIResized = cv2.resize(
            imgROI, (RESIZED_IMAGE_WIDTH, RESIZED_IMAGE_HEIGHT))

        # flatten image into 1d numpy array
        ROIResized = imgROIResized.reshape(
            (1, RESIZED_IMAGE_WIDTH * RESIZED_IMAGE_HEIGHT))

        # convert from 1d numpy array of ints to 1d numpy array of floats
        ROIResized = np.float32(ROIResized)

        retval, Results, neigh_resp, dists = kNearest.findNearest(
            ROIResized, k=1)     # call KNN function to findnearest

       
        strCurrentChar = str(chr(int(Results[0][0])))

        
        string_output = string_output + strCurrentChar

    print("\n" + 'Recognized character output:', string_output + "\n")

    with open('recognized_output.txt', 'a+') as text_file:
        print('Recognized character output:', string_output, file=text_file)

    # show input image with border boxes drawn around found digits
    cv2.imshow("Input Test Image", inputTestingImage)
    cv2.waitKey(0)

    cv2.destroyAllWindows()
    return


In [5]:
if __name__ == "__main__":
    main()



Recognized character output: MUHAMMADSAAD92

